# Lesson 16 深度学习入门(下)

## 一、架构对学习能力/鲁棒性的影响

### 1.深度

#### 1.1 困难和实践

深度学习普遍认知： 更深的网络会展示出更强大的学习能力

**深度**:卷积神经网络当中，带有权重的层的数量，也指全部层的数量。通常来说，深度越深，参数量越大，卷积网络的规模越大。

- 输入图像的尺寸会限制我们可以选择的深度
- PyTorch中，池化层的padding的参数值必须小于池化核尺寸的1/2，否则程序会报错
- 卷积层的padding参数值虽然不受程序强制限制，但一般也会设置为至少是小于卷积核尺寸
  - 当kernel=padding时，卷积层会增加不必要的计算，甚至会单纯增加噪音

我们常见的特征图尺寸变化其实只有以下三种：    
1. 宽高分别折半、或缩小更多：当步长为2以上，或者卷积核较大时，可以实现大幅度缩小特征图的尺寸。
2. 不使用池化层，利用填充与卷积核尺寸的搭配，令特征图每经过一次卷积层，就缩小2个或4个像素，
3. 利用填充与卷积核尺寸的搭配，令特征图每经过一次卷积层，尺寸不变，将缩小特征图的工作完全交给池化层来做






**VGGNet:**
- 核心思想就是使用多个连续且保持特征图尺寸不变的卷积层来增加深度，重复架构
- 每重复几个卷积就会跟一个最大池化层，这种“n个卷积+池化”的结构在VGG中被称之为“块”（block）

随着深度的加深，模型的学习能力大概率会增强，但深度与模型效果之间的关系不是线性的，可以增长的边际准确率是在递减的，准确率的变化会逐渐趋于平缓。对于复杂的数据集，神经网络的层数都达到了200+

### 1.2 VGG16的复现

VGG架构对于神经网络研究和使用都有重要的意义，它不仅简单、有效，而且非常适合用来做各种实验和测试。在我们已经详细复现AlexNet与LeNet5的基础上，VGG架构的代码就显得异常简单。在这里，我为大家提供输入为224x224的VGGNet16的详细架构和复现后的代码，大家可以参考。

VGG16的架构用语言来表示则有：  
 
**输入→（卷积x2+池化）x2 →（卷积x3+池化）x3 → FC层x3 →输出**

1. 除了输出层外，所有的激活函数都是ReLU函数  
2. 最后三个全连接层中的前两个全连接层前有Dropout层，p=0.5 
3. 每一次卷积后面都要加上激活函数 ,每个线性层后面也需要激活函数

![](https://gitee.com/bravojimoon/note-picture/raw/master/torch/4D4H1AZ5A%7DUHLI@%5B9$1A%60SW.png)


In [1]:
from doctest import OutputChecker
from turtle import forward
from torch import conv2d, relu, softmax


class VGG16(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        
        # block1
        self.conv1 = nn.Conv2d(3,64,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2)
        # block2
        self.conv3 = nn.Conv2d(64,128,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(128,128,kernel_size=3,padding=1)
        self.pool2 = nn.MaxPool2d(2)

        # block3
        self.conv5 = nn.Conv2d(128,256,kernel_size=3,padding=1)
        self.conv6 = nn.Conv2d(256,256,kernel_size=3,padding=1)
        self.conv7 = nn.Conv2d(256,256,kernel_size=3,padding=1)
        self.pool3 = nn.MaxPool2d(2)

        # block4
        self.conv8 = nn.Conv2d(256,512,kernel_size=3,padding=1)
        self.conv9 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.conv10 = nn.Conv2d(512,512,kernel_size=3,padding=1)
        self.pool4 = nn.MaxPool2d(2)
        
        # block5
        self.conv11 = nn.Conv2d(512,512,3,padding=1)
        self.conv12 = nn.Conv2d(512,512,3,padding=1)
        self.conv13 = nn.Conv2d(512,512,3,padding=1)
        self.pool5 = nn.MaxPool2d(2)

        # fc
        self.fc1 = nn.Linear(7*7*512,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,10)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool3(F.relu(self.conv7(x)))

        x = F.relu(self.conv8(x))
        x = F.relu(self.conv9(x))
        x = self.pool4(F.relu(self.conv10(x)))

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = self.pool5(F.relu(self.conv13(x)))

        x = x.view(-1,7*7*512)

        x = F.relu(self.fc1(F.dropout(x,p= 0.5)))
        x = F.relu(self.fc2(F.dropout(x,p= 0.5)))
        x = F.relu(self.fc3(x))

        output = F.softmax(x,dim=1)
        return output



In [2]:
from torchinfo import summary

# 实例化
vgg = VGG16()
summary(vgg, input_size=(10, 3, 224, 224),device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [10, 10]                  --
├─Conv2d: 1-1                            [10, 64, 224, 224]        1,792
├─Conv2d: 1-2                            [10, 64, 224, 224]        36,928
├─MaxPool2d: 1-3                         [10, 64, 112, 112]        --
├─Conv2d: 1-4                            [10, 128, 112, 112]       73,856
├─Conv2d: 1-5                            [10, 128, 112, 112]       147,584
├─MaxPool2d: 1-6                         [10, 128, 56, 56]         --
├─Conv2d: 1-7                            [10, 256, 56, 56]         295,168
├─Conv2d: 1-8                            [10, 256, 56, 56]         590,080
├─Conv2d: 1-9                            [10, 256, 56, 56]         590,080
├─MaxPool2d: 1-10                        [10, 256, 28, 28]         --
├─Conv2d: 1-11                           [10, 512, 28, 28]         1,180,160
├─Conv2d: 1-12                           [10, 5

### 2.感受野

#### 2.1 认识感受野

另一个常常被认为是与卷积神经网络的预测效果相关的因素是感受野的尺寸。

- 稀疏交互：每一个神经元只受上一层的部分神经元的影响（相当与不是全连接的）  
  
- **感受野**  
在深度卷积神经网络中，每个神经元节点都对应着输入图像上的某个区域，且该神经元仅受这个区域中的图像内容的影响，那么这个区域称之为神经元的感受野。

#### 2.2 认识感受野的性质

- 深度越大，感受野越大，池化层放大感受野的效率更高
- 放大感受野，是否有极限
  - 按理来说感受野有上限（图像的大小），但实际上没有上限，感受野越大越好！！
    - 感受野大小比图像大小还大时，图像就会出现黑边，增加了噪音 -> 关注中心，周围模糊
- 关注中心，周围模糊
  - 位于图像中间的像素有更多可以影响最终特征图的“路径”，他们对最终特征图的影响更大，对卷积网络的分类造成的影响也会更大。


#### 2.3 扩大感受野：膨胀卷积

膨胀卷积就是在每个参与卷积计算的计算点上做“膨胀”操作，让计算点与计算点之间出现“空洞”，并跳过空洞进行计算的卷积方式。
- 膨胀卷积不适合小物体的分割
- 详细见PDF

#### 2.4 感受野尺寸的计算

对于第$l$个卷积层/池化层输出的特征图而言，假设卷积核尺寸为正方形、输入架构的图像尺寸也为正方形，则该图上任意一个神经元的感受野大小为：
$$r_l = r_{l-1}+(k_l-1)*\prod_{i=0}^{l-1}{s_i}$$

- $r_{l-1}$为上一个卷积层/池化层的感受野的大小， 
- $k_l$这一层的卷积核/池化核的大小，
- $s_i$是第$i$层的卷积/池化步长。  
  
不难发现，在这个公式的最后一部分，是$l$层之前所有对感受野有影响的卷积、池化层的步长的连乘结果，可见，感受野的大小只与卷积核的大小、各层的步长有关，感受野的大小与padding无关。

- 全连接层不需要计算感受野
- pytorch中自动进行感受野计算的包torch_receptive_field

In [3]:
data = torch.ones(size=(10,1,32,32))

In [4]:
#————————————————————————————例子:计算LeNet和AlexNet感受野的尺寸————————————————————————————
# 两个网络的架构在上一节已经实现过了


# 每一层感受野的计算公式：
# 这一层的感受野 = 上一层的感受野的尺寸 + (这一层核的尺寸-1 )*连乘(从最初一层到上一层的步长)

class LeNet5(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,5)                        # l0=1  1+(5-1)*1 = 5
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)    # 5+(2-1)*(1*1) =  6 
        #这里更换为maxpool，在原论文中为average pool
        self.conv2 = nn.Conv2d(6,16,5)                       # 6 + (5-1)*(1*1*2) = 14  # 上一层的步长为2
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)    # 14 +(2-1)*(1*1*2*1) = 16
        self.fc1 = nn.Linear(16*5*5,120)                     # 线性层不影响感受野的大小
        self.fc2 = nn.Linear(120,84)
    
    def forward(self,x):
        x = F.tanh(self.conv1(x))
        x = self.pool1(x)
        x = F.tanh(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,16*5*5)
        x = F.tanh(self.fc1(x))
        output = F.softmax(self.fc2(x),dim=1)
        output = F.softmax(x.view(-1,16*5*5),dim=1)

 
        
data = torch.ones(size=(10,3,227,227)) #假设图像的尺寸为227x227

class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        #大卷积核、较大的步长、较多的通道
        self.conv1 = nn.Conv2d(3,96,kernel_size=11, stride=4)    # 1 + (11-1)*1 = 11
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=2)        # 11 + (3-1)*1*4 = 19
        
        #卷积核、步长恢复正常大小，进一步扩大通道
        self.conv2 = nn.Conv2d(96,256,kernel_size=5, padding=2)  # 19 +(5-1)*(1*4*2) = 51
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=2)        # 51 + (3-1)*(8) = 67
        
        #连续的卷积层，疯狂提取特征
        self.conv3 = nn.Conv2d(256,384,kernel_size=3,padding=1) 
        self.conv4 = nn.Conv2d(384,384,kernel_size=3,padding=1)
        self.conv5 = nn.Conv2d(384,256,kernel_size=3,padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=3,stride=2)



### 3.平移不变性

**不变性**：如果我们能够识别出一张图像中的一个对象，那即便这个对象以完全不同的姿态呈现在别的图像中，我们依然可以识别出这个对象。对算法而言，不变性意味着在训练集上被成功识别的对象，即便以不同的姿态出现在测试集中，也应该能够被成功识别。

**鲁棒性（robustness）**:
- 衡量一个系统所具有的抵抗变化、使自身保持稳定的能力
  - 在数据科学中，鲁棒性用于形容一个模型或算法在不同数据、不同环境下的表现是否稳定
  - 特别地，一个过拟合的模型的鲁棒性一定是较低的，因为过拟合就意味着不能适用数据的变化
- 大部分深层卷积网络的架构自带一定的“平移不变性”

池化层只能解决平移不变性的问题，对其他不变性却无计可施，为了让卷积神经网络具备各类不变性，我们需要采取更强力的手段：数据增强（Data Augmentation）。

**数据增强**是数据科学体系中常用的一种增加数据量的技术，它通过添加略微修改的现有数据、或从现有数据中重新合成新数据来增加数据量。

## 二、架构对参数量/计算量的影响

深度学习模型天生就需要大量数据进行训练，因此每次训练中的参数量和计算量就格外关键，因此在设计卷积网络时，我们希望**相似预测效果下，参数量越少越好**。

对于卷积神经网络中的任意元素（层或函数），有两种方式影响模型的参数量：  
1. 这个层自带参数，其参数量与该层的超参数的输入有关  
2. 这个层会影响feature map的尺寸，影响整体像素量和计算量，从而影响全连接层的输入  

全连接层、bn通过第一种方式影响参数，而池化、padding、stride等操作则通过第二种方法影响参数，卷积层通过两种方式影响参数，dropout、激活函数等操作不影响参数量。

### 1.卷积层

#### 1.1 参数量计算

一个卷积网络的卷积层究竟包含多少参数量，就是由卷积核的尺寸kernel_size、输入的通道数in_channels，输出的通道数out_channels（卷积核的数量）共同决定的。其参数量如下：
$$N_{parameters}=(K_H*K_W*C_{in})*C_{out}+C_{out}$$

其中，加号前面的部分是权重$w$的数量，加号之后的是偏置$b$的数量。
- padding以及stride这些参数，不影响卷积层的所需要的参数量
- 卷积核的尺寸都比较小，真正影响参数量的是输入的参数和输入的特征图的大小
- `.numel()` torch中用与计数的函数



In [6]:
#————————————————————————————例子：计算参数————————————————————————————

# 实例化 - 实例化的瞬间就会生成随机的参数
conv1 = nn.Conv2d(3,6,3)   # 3*3*3*6+6
conv2 = nn.Conv2d(6,4,3)   # 3*3*6*4 +4

conv1.weight.numel()
conv1.bias.numel()

# numel() - tensor中计数的函数


conv2.weight.numel()
conv2.bias.numel()


162

6

216

4

In [7]:
#——————————————————————————padding以及stride这些参数，不影响卷积层的所需要的参数量———————————————————————————
conv3 = nn.Conv2d(4,16,5,stride=2,padding=1)  # 5*5*4*16+16 = 1616  = 1600 +16

conv4 = nn.Conv2d(16,3,5,stride=3,padding=2)  # 5*5*16*3+3  
# (5*5*16)*3 + 3

conv3.weight.numel()
conv3.bias.numel()

conv4.weight.numel()
conv4.bias.numel()



1600

16

1200

3

#### 1.2 大尺寸卷积核vs小尺寸卷积核

- 大尺寸卷积核的效果可由多个小尺寸卷积核累积得到，默认都是用小卷积核

#### 1.3 1x1卷积核
在众多的小卷积核中，小到极致的就是1x1尺寸的卷积核。
- 加深CNN的深度
- 用两个卷积层之间，用于调整输出的通道数，协助大幅度降低计算量和参数量，从而协助加深网络深度，**跨通道信息交互**
  - 下面两种方法都使用1*1卷积核：
  - 方法1.`conv(256,256,3)`
    - 需要$3*3*256*256+256=590080$个参数,
  - 方法2.利用`conv(256,32,1)->conv(32,32,3)->conv(32,256,1) `
    - 需要$(256*32+32)+(3*3*32*32+32)+(32*256*1+256)=25920$个参数 
  - 参数量大大的缩减

 
- **瓶颈设计（bottleneck design）**:在核尺寸为1x1的2个卷积层之间包装其他卷积层的架构
  - 瓶颈设计常用于多于100层以上的网络中

1x1的卷积核上只有一个权重，每次进行卷积操作时，该权重会与原始图像中每个像素相乘，并得到特征图上的新像素，因此1x1卷积也被叫做“逐点卷积”（Pointwise Convolution）,这种计算方式和矩阵*常数一致


#### 1.4 减少参数量：分组卷积与深度分离卷积

- 参数`groups`设置分组数,一般为偶数；将输入的特征图和输出的特征图都分成g组
- 分组卷积也是一种高效降低参数数量的形式,通过给输入特征图及输出特征图分组来消减连接数量的卷积方式
- 分组数g一般为偶数，将输入的特征图和输出的特征图都分成g组
- 分组的存在不影响偏置，偏置只与输出的特征图数量有关


例如：
  - 不分组: conv(4,8,3) 
    -  参数：$3*3*4*8 =288$
  - 分组： conv(4,8,3,g=1) & conv(4,8,3,g=2) 
    - 参数: $3*3*2*4 + 3*3*2*4 =144$
     - 4个输入8个输出分成两组，每一组就是2个输入和4个输出
  - 参数数量减少50% 





In [8]:
#————————————————————————————例子：分组卷积————————————————————————————
conv1 = nn.Conv2d(4,8,3) #(3 * 3 * 4)*8 + 8 = 296
conv1_ = nn.Conv2d(4,8,3,groups=2) # ((3 * 3 * 4)*8)/2 + 8 = 152

#检查一下结果
conv1.weight.numel()
conv1.bias.numel()

conv1_.weight.numel()
conv1_.bias.numel()

288

8

144

8

In [9]:
# 输入奇数
conv2 = nn.Conv2d(4,8,3,groups=3)

# 直接报错：in_channels must be divisible by groups
# 输入输出的数据 要能够被groups除尽

ValueError: in_channels must be divisible by groups

- 深度卷积
  - groups参数最大可以取到和$C_{in}$或$C_{out}$中较小的那个值一样大。
  - **groups =$C_{in}$的分组卷积叫做“深度卷积”**（Depthwise Convolution）
  - 比起普通卷积，参数量是原来的$\frac{1}{C_{in}}$  


- 深度卷积与1x1卷积核结合使用  
我们首先进行深度卷积，产出一组特征图，然后再这组特征图的基础上执行1x1卷积，对特征图进行线性变换。两种卷积打包在一起成为一个block，这个block就叫做“深度可分离卷积”（Depthwise separable convolution）
对于深度可分离卷积的一个block，若不考虑偏置，则整个block的参数量为：
$$parameters = K_H*K_W*C_{out}^{depth}+C_{in}^{pair}*C_{out}^{pait}$$

若原始卷积也不考虑偏置，则深度可分离卷积的参数比上原始卷积的参数的比例为：
  $$ratio = \frac{C_{out}^{depth}}{C_{in}^{depth}*C_{out}^{pair}}+\frac{C_{in}^{pair}}{K_H*K_w*C_{in}^{depth}}$$

In [10]:
# 普通卷积： (ks^2*c_in)*c_in
# 分组卷积: 1/g*(ks^2*C_in * C_out), 深度卷积就是g= C_in
# 深度可分离卷积： ks^2*C_in_depth + C_in_pair* C_in_pair
# 比例： 1/C_in_depth + C_out_pair /(ks^2*C_in_depth)


In [11]:
#————————————————————————————普通卷积————————————————————————————
#与图上不同，在代码中我们令输出的特征图数量与输入的特征图数量不相等，用以区别。输出特征图数量=8。
conv1 = nn.Conv2d(4,8,3, bias=False) # (3 * 3 * 4) * 8 = 288


In [12]:
#————————————————————————————深度可分离卷积————————————————————————————
# 深度卷积 groups = C_in
conv_depthwise = nn.Conv2d(4,8,3,groups=4,bias=False) # 1/4 * (3 * 3 * 4)*8 = 72

# 逐点卷积 kernel_size = 1
conv_pairwise = nn.Conv2d(8,8,1,bias=False) # 64

In [13]:
#————————————————————————————验证比例公式————————————————————————————
# 利用比例公式计算
1/4 + 8/(3*3*4)

# 提取参数计算比例
(conv_depthwise.weight.numel()+conv_pairwise.weight.numel())/(conv1.weight.numel())

0.4722222222222222

0.4722222222222222

### 2.全连接层

真正对CNN参数量“贡献”巨大的是全连接层。数据在进入全连接层时，需要将所有像素拉平，而全连接层中的一个像素点就对应着一个参数，因此全连接层所携带大量参数。为什么卷积网络里需要有全连接层呢？全连接层的作用主要有以下两个：

**1. 作为分类器，实现对数据的分类**

卷积层提供了一系列有意义且稳定的特征值，构成了一个与输入图像相比维数更少的特征空间，而全连接层负责学习这个空间上的（可能是非线性的）函数关系，并输出预测结果。

**2、作为整合信息的工具，将特征图中的信息进行整合**

基于上面的两个理由，我们一般都会在形似AlexNet或VGG的网络中包含全连接层。但一旦有可以替换全连接层、并不影响模型效果的手段，大家就会尝试将全连接层替代掉。

对于CNN中的全连接层来说，在一个层上增加足够多的神经元，会比增加层效果更好。一般来说，CNN中的全连接层最多只有3-4层（包括输出层），过于多的层会增加计算的负担，还会将模型带入过拟合的深渊。对于小型网络，3层全连接层已是极限了。

需要注意的是，在卷积层和全连接层的连接中，通常全连接的输出神经元个数不会少于输入的通道数。对于全连接层之间的连接，只要不是输出层，也很少出现输出神经元少于输入神经元的情况。


#### 2.1 从卷积到全连接层

决定全连接层参数数量的有两个因素：最后一个卷积层上的特征图所含的像素量，以及我们在全连接层之间设定的输出神经元个数。

-  **如何找出最后一个池化层/卷积层上输出的特征图的尺寸呢？**
一种简单的方法是，将Model中所有的线性层都注释掉，只留下卷积层，然后将model输入summary进行计算，但有更简单的方法，使用另一种构筑神经网路架构的方式：nn.Sequential。  

`nn.Sequential`是一种非常简单的构筑神经网络的方式，它可以将“以序列方式从前往后运行的层”打包起来，组合成类似于机器学习中的管道（Pipeline）的结构.

- 代码量明显较少，但是卷积层的架构不太明显
- 使用`nn.Sequential`来调试卷积层架构，并不断查看感受野的变化。
- `nn.Sequential`中无须写明激活函数 

In [14]:
#————————————————————————————使用nn.Squential架构网络————————————————————————————

data = torch.ones(size=(10,3,229,229))

#不使用类，直接将需要串联的网络、函数等信息写在一个“序列”里
#重现上面的4个卷积层、2个池化层的架构

net = nn.Sequential(nn.Conv2d(3,6,3),
                    nn.ReLU(inplace=True)  # 实行ReLU以后，立即生效
                    ,nn.Conv2d(6,4,3)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                    ,nn.Conv2d(4,16,5,stride=2,padding=1)
                    ,nn.ReLU(inplace=True)
                    ,nn.Conv2d(16,3,5,stride=3,padding=2)
                    ,nn.ReLU(inplace=True)
                    ,nn.MaxPool2d(2)
                    )


#nn.Sequential组成的序列不是类，因此不需要实例化，可以直接输入数据
net(data).shape

# 3个特征图每个是9*9


# 调试神经网络的时候使用，不需要专门定义一个类
# 不需要写入线性层

torch.Size([10, 3, 9, 9])

In [15]:
#————————————————————————————利用nn.Squential查看输入到全连接层的输入数————————————————————————————

net = nn.Sequential(nn.Conv2d(3,64,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(64,64,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(64,128,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(128,128,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(128,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(256,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),        
                                        )

data = torch.ones(size=(10,3,224,224))

net(data).shape

# 512个特征图，尺寸为7*7

torch.Size([10, 512, 7, 7])

- VGG16架构
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/7.png?versionId=CAEQFRiBgMD2zKyfxxciIGZhMDQ0Y2UyYTA5ZjQ1NjhhMWNjNDQ1Njg3YTFiODZh)

In [16]:
#————————————————————————————VGG16为例，使用nn.Sequential的架构————————————————————————————

class VGG16(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        # 特征提取
        self.features_ = nn.Sequential(nn.Conv2d(3,64,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(64,64,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(64,128,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(128,128,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(128,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(256,256,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(256,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),

                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.Conv2d(512,512,3,padding=1),nn.ReLU(inplace=True),
                                       nn.MaxPool2d(2),        
                                        )
        # 分类器
        self.clf_ = nn.Sequential(nn.Dropout(p=0.5),
                                nn.Linear(512*7*7,4096),nn.ReLU(inplace=True),  # 512*7*7 在上一个cell中利用nn.Squential计算的
                                nn.Dropout(p=0.5),
                                nn.Linear(4096,4096,nn.ReLU(inplace=True)),
                                nn.Linear(4096,1000),nn.Softmax(dim=1))

    def forward(self,x):
        x = self.features_(x)  # 用于特征提取的架构提取特征
        x = x.view(-1,512*7*7)  # 调整数据结构，拉平函数
        output = self.clf_(x)
        return output


In [17]:
vgg = VGG16()  # 实例化定义好的类

In [18]:
from receptive_field import receptive_field
from torchinfo import summary

summary(vgg,input_size=(10,3,224,224),device='cpu')  # 会自动调用GPU

# 没有全部显示完全


Layer (type:depth-idx)                   Output Shape              Param #
VGG16                                    [10, 1000]                --
├─Sequential: 1-1                        [10, 512, 7, 7]           --
│    └─Conv2d: 2-1                       [10, 64, 224, 224]        1,792
│    └─ReLU: 2-2                         [10, 64, 224, 224]        --
│    └─Conv2d: 2-3                       [10, 64, 224, 224]        36,928
│    └─ReLU: 2-4                         [10, 64, 224, 224]        --
│    └─MaxPool2d: 2-5                    [10, 64, 112, 112]        --
│    └─Conv2d: 2-6                       [10, 128, 112, 112]       73,856
│    └─ReLU: 2-7                         [10, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [10, 128, 112, 112]       147,584
│    └─ReLU: 2-9                         [10, 128, 112, 112]       --
│    └─MaxPool2d: 2-10                   [10, 128, 56, 56]         --
│    └─Conv2d: 2-11                      [10, 256, 56, 56]         29

#### 2.2 代替全连接层：1x1卷积核与全局平均池化（GAP）

- 是否能用1x1卷积核来进行替代全连接层?
  - 可以，但是使用效果微乎及微
  - 参数量没有减少...
  - 最大的好处就是解放了输入层对图像尺寸的限制...
    - 在拉平进入到输入层中，要求具体的输入数
    - 如果全部是卷积层就不需要拉平


对于卷积层来说，只要让特征图的尺寸为1x1，再让卷积核的尺寸也为1x1，就可以实现和普通全连接层一模一样的计算了。
- 不包含全连接层，只有卷积层和池化层的卷积网络被叫做全卷积网络（fullyconvolutional network，FCN）。


- NiN网络中用全局平均池化GAP代替了全连接层
  - GAP层的本质是池化层，它使用池化方式是平均池化，它的职责就是将上一层传入的无论多少特征图都转化成(n_class,1, 1)结构。

### 3.NiN网络的复现

- NiN网络使用1x1卷积层(实际上是GAP层)代替的全连接层

我们就使用nn.Sequential来打包实现一下NiN网络。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/58.png?versionId=CAEQFRiBgIC0zIWfxxciIGE0ZjkyMzI1ZTE3MzQ3MmQ5NDhiZWFlN2UyMmFiYTQ0)

In [19]:
# 输入数据的尺寸
data = torch.ones(10,3,32,32)

In [20]:
from torch import dropout

# 为什么要把特征图的尺寸 先放大在放下？

class NiN(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3,192,5,padding=2),nn.ReLU(inplace=True),
            nn.Conv2d(192,160,1),nn.ReLU(inplace=True),
            nn.Conv2d(160,96,1),nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),   # 步长小于核尺寸 -> 重叠池化
            nn.Dropout(p=0.25)  # 未明确给出，但是卷积层的的p值一般在0.25以下，线性层的p值给的比较大0.5 
        )
        self.block2 =nn.Sequential(
            nn.Conv2d(96,192,5,padding=2),nn.ReLU(inplace=True),
            nn.Conv2d(192,192,1),nn.ReLU(inplace=True),
            nn.Conv2d(192,192,1),nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3,stride=2),
            nn.Dropout(p=0.25)
        )
        self.block3 =nn.Sequential(
            nn.Conv2d(192,192,3,padding=1),nn.ReLU(inplace=True),
            nn.Conv2d(192,192,1),nn.ReLU(inplace=True),
            nn.Conv2d(192,10,1),nn.ReLU(inplace=True),
            nn.AvgPool2d(7,stride=1),   # 全局平均池化
            nn.Softmax(dim=1)
        )

    def forward(self,x):
        output = self.block3(self.block2(self.block1(x)))
        return output

In [21]:
# 实例化 
net = NiN()
net(data).shape

torch.Size([10, 10, 1, 1])

In [22]:
summary(net,input_size=(10,3,32,32))

# 在Sequential中分开定义以后，summary也分层展示

#作为9层卷积层、最大特征图数目达到192的网络，NiN的参数量在百万之下，可以说都是归功于没有使用全连接层。

Layer (type:depth-idx)                   Output Shape              Param #
NiN                                      [10, 10, 1, 1]            --
├─Sequential: 1-1                        [10, 96, 15, 15]          --
│    └─Conv2d: 2-1                       [10, 192, 32, 32]         14,592
│    └─ReLU: 2-2                         [10, 192, 32, 32]         --
│    └─Conv2d: 2-3                       [10, 160, 32, 32]         30,880
│    └─ReLU: 2-4                         [10, 160, 32, 32]         --
│    └─Conv2d: 2-5                       [10, 96, 32, 32]          15,456
│    └─ReLU: 2-6                         [10, 96, 32, 32]          --
│    └─MaxPool2d: 2-7                    [10, 96, 15, 15]          --
│    └─Dropout: 2-8                      [10, 96, 15, 15]          --
├─Sequential: 1-2                        [10, 192, 7, 7]           --
│    └─Conv2d: 2-9                       [10, 192, 15, 15]         460,992
│    └─ReLU: 2-10                        [10, 192, 15, 15]         -

作为9层卷积层、最大特征图数目达到192的网络，NiN的参数量在百万之下，可以说都是归功于没有使用全连接层。不过，1x1卷积层所带来的参数量也不少，因此NiN可以说是在各方面都中规中矩的网络。

受到NiN网络启发而诞生的GoogLeNet以及ResNet都使用了1x1卷积层，并且在各种消减参数的操作下使网络变得更加深。

## 三、前沿网络state-of-the-art models

在深度学习的领域，最前沿、最先进的架构被称为state-of-the-art models，简写为SOTA，我将其翻译为“前沿网络”。

### 1.GoogLeNet(Inveption V1)

#### 1.1 动机与思路

VGG的参数过,各层之间的链接过于“稠密”（Dense），计算量过大，并且很容易过拟合。为了解决这个问题，我们之前已经提出了多种方法，其中最主流的是：  
1. 使用我们在上一节中提出的分组卷积、舍弃全连接层等用来消减参数量的操作，让神经元与神经元之间、或特征图与特征图之间的连接数变少，从而让网络整体变得“稀疏”
2. 引入随机的稀疏性。例如，使用类似于Dropout的方式来随机地让特征矩阵或权重矩阵中的部分数据为0
3. 引入GPU进行计算

在2014年之前，以上操作就是我们目前为止接触的所有架构在减少参数量、防止过拟合上做出的努力。其中NiN主要使用方法1，AlexNet和VGG主要使用方法2和3，但这些方法其实都存在一定的问题：

首先，**分组卷积等操作虽然能够有效减少参数量，却也会让架构的学习水平变得不稳定**。在神经网络由稠密变得稀疏（Sparse）的过程中，网络的学习能力会波动甚至会下降。

其次，**随机的稀疏性与GPU计算之间其实是存在巨大矛盾的**。现代硬件不擅长处理在随机或非均匀稀疏的数据上的计算，并且这种不擅长在矩阵计算上表现得尤其明显。



- GoogLeNet团队的思路是：  
使用普通卷积、池化层这些稠密元素组成的块去无限逼近一个稀疏架构，从而构造一种参数量与稀疏网络相似的稠密网络。这种思路的核心不是通过减少连接、减少扫描次数等“制造空隙”的方式来降低稠密网络的参数量，而是直接在架构设计上找出一种参数量非常少的稠密网络。

基于这样的基本理念，GoogLeNet团队使用了一个复杂的网络架构构造算法，并让算法向着“使用稠密成分逼近稀疏架构”的方向进行训练，产出了数个可能有效的密集架构。在进行了大量的实验之后，他们选出了学习能力最强的密集架构及其相关参数，这个架构就是Inception块和GoogLeNet。


#### 1.2 Inception V1

**Inception**块使用了卷积层、池化层并联的方式。在一个Inception块中存在4条线路，每条线路可以被叫做一个分枝（branch）：
1. 第一条线路上只有一个1x1卷积层，只负责降低通道数
2. 第二条路线由一个1x1卷积层和一个3x3卷积层组成，本质上是希望使用3x3卷积核进行特征提取，但先使用1x1卷积核降低通
道数以此来降低参数量和计算量（降低模型的复杂度）
3. 第三条线路由一个1x1卷积层和一个5x5卷积层组成，其基本思路与第二条线路一致
4. 最后一条线路由一个3x3池化层和一个1x1卷积层组成，将池化也当做一种特征提取的方式,并在池化后使用1x1卷积层来降低通道数。
5. 不难注意到，所有的线路都使了巧妙的参数组合，让特征图的尺寸保持不变，因此在四条线路分别输出结果之后，Inception块将四种
方式生成的特征图拼接在一起，形成一组完整的特征图，

![](https://gitee.com/bravojimoon/note-picture/raw/master/torch/%5D@1BCKVD9X$1%5DI@OQ91RCC9.png)

优势：
- 首先，同时使用多种卷积核可以确保各种类型和层次的信息都被提取出来。
- 其次，并联的卷积池化层计算效率更高。
- 大量使用1x1卷积层来整合信息，既实现了“聚类信息”又实现了大规模降低参数量，让特征图数量实现了前所未有的增长。  
- Softmax作为主分类器，还有两个辅助分类器
- GoogLeNet集成了两个浅层网络和一个深层网络的结果来进行学习和判断，在一个架构中间增加集成的思想

  **辅助分类器**   
  
GoogLeNet还使用了“辅助分类器”（auxiliary classifier）以提升模型的性能。辅助分类器是除了主体架构中的softmax分类器之外，另外存在的两个分类器。在整体架构中，这两个分类器的输入分别是inception4a和inception4d的输出结果
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/辅助分类器.PNG?versionId=CAEQFRiBgIC7w62XyhciIGNkN2E5ZGRiMjlmZTQxYzY5YmY1ODA3YWI0YjYyNDYx)

Google团队在迭代中加重中层inception输出的特征的权重，就可能将模型引导向更好的反向。因此，他们将位于中间的inceptions的结果使用辅助分类器导出，并让两个辅助分类器和最终的分类器一共输出三个softmax结果、依次计算三个损失函数的值，并将三个损失加权平均得到最终的损失。

#### 1.3 GoogLeNet的复现

GoogLeNet是我们遇见的第一个串联元素中含有更多复杂成分的网络，因此我们需要先单独定义几个单独的元素，之后才能够使用我们熟悉的建立类的方式来复现架构。


- 定义：基础卷积层

首先，能够在主体网络中省略掉所有的激活函数，我们需要定义新的基础卷积层。这个卷积层是包含激活函数以及BN层的卷积层。这样定义能够帮助我们大幅度减少最后在整合好的GoogLeNet中会出现的ReLU函数以及BN层。


![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/GoogleNet.PNG?versionId=CAEQFRiBgIDNw62XyhciIDA0ZDViOWM5YjYyNDRlZmJiMmMzNWVjOTZlNjk5YmMw)

上图是GoogLeNet的主体架构。Inception内部是稠密部件的并联，而整个GoogLeNet则是数个Inception块与传统卷积结构的串联。

- 深度学习中的神经网络bias一般为False
- nn.Sequential中需要写明激活函数
- 利用self.架构神经网络中不需要写激活函数，只需要在forward函数中定义数据的流向就行


In [23]:
# conv + BN + ReLU --> basicConv
# Incepetion
# AUXclcf

import torch
from torchinfo import summary
from torch import nn

In [10]:
#————————————————————————————定义：基础的卷积层 - 包含激活函数和BN层————————————————————————————

class BasicConv2d(nn.Module):
    #在这里我们要改掉原来会输入参数默认值的习惯，而使用定义类型的方式，同时将**kwargs也放到init中继承
    #**kwargs代表了“所需要的全部参数”，由于现在的架构变得复杂，我们不太可能将每个需要用的参数都写在定义中
    #因此，我们继承**kwargs来获得所需类的全部参数
    def __init__(self,in_channels:int,out_channels:int,**kwargs) -> None:
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels,out_channels,bias=False,**kwargs),
                                nn.BatchNorm2d(out_channels),
                                nn.ReLU(inplace=True)
        )
    
    def forward(self,x):
        output = self.conv(x)
        return output


#测试
BasicConv2d(2,10,kernel_size=3) #这里的输入数据是随意输入的，只是为了我们所写的类能够跑通

# 试试看如果不写**kwargs，会发生什么？ - 报错

BasicConv2d(
  (conv): Sequential(
    (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

- 定义：Inception块
  
由于Inception块中是并联的结构，存在4个branchs，所以我们不能使用nn.Sequential进行打包，而是要使用原始的self.的形式。在Inception块中，所有卷积、池化层的输入、输出以及核大小都需要我们进行输入。Inception结构如下：

![](https://gitee.com/bravojimoon/note-picture/raw/master/torch/%5D@1BCKVD9X$1%5DI@OQ91RCC9.png)

- 4个分支最大不同就是输出的不同,通过参数传入
- 3x3 reduce和5x5 reduce就是指inception块中3x3和5x5卷积层之前的1x1卷积层的输出量
- pool proj中写的数字实际上是池化层后的1x1卷积层的输出量。


In [11]:
#————————————————————————————定义Inception块————————————————————————————
class Inception(nn.Module):
    """
    所有的数据都要通过这4个分支
    """
    def __init__(self
                ,in_channels :int
                ,ch1x1 : int
                ,ch3x3red : int
                ,ch3x3 : int
                ,ch5x5red : int
                ,ch5x5 : int
                ,pool_proj) -> None:   # 6个参数 属于6个卷积层
        super().__init__()
        
        #  branch1 : 1x1
        self.branch1 = BasicConv2d(in_channels,ch1x1,kernel_size = 1)
        
        # brach2 : 1x1 + 3x3
        self.branch2 = nn.Sequential(BasicConv2d(in_channels,ch3x3red,kernel_size=1),
                                     BasicConv2d(ch3x3red,ch3x3,kernel_size=3,padding=1))
        
        # 1x1+5x5
        self.branch3 = nn.Sequential(BasicConv2d(in_channels,ch5x5red,kernel_size=1),
                                     BasicConv2d(ch5x5red,ch5x5,kernel_size=5,padding=2))
        
        # pool + 1x1
        self.branch4 = nn.Sequential(nn.MaxPool2d(kernel_size=3,stride=1,padding=1,ceil_mode=True),  # 池化层不改变数据的尺寸
                                     BasicConv2d(in_channels,pool_proj,kernel_size=1))
    
    # 数据流是并联 - 最后合并在一起
    def forward(self,x):
        branch1 = self.branch1(x)  # 28x28,ch1x1
        branch2 = self.branch2(x)  # 28x28,ch3x3
        branch3 = self.branch3(x)  # 28x28,ch5x5
        branch4 = self.branch4(x)  # 28x28,ch5x5
        outputs = [branch1, branch2, branch3, branch4]   #  (28x28,ch1x1+ch3x3+ch5x5+ch5x5)
        return torch.cat(outputs,1)  # 水平方向上合并

In [12]:
# 测试

data = torch.ones(10,192,28,28)

in3a = Inception(192,64,96,128,16,32,32) #这是inception3a的参数

in3a(data).shape

torch.Size([10, 256, 28, 28])

- 定义：辅助分类器
  
接下来，还需要单独定义的是辅助分类器（Auxiliary Classifier）的类。辅助分类器的结构其实与我们之前所写的传统卷积网络很相似，因此我们可以使用nn.Sequential来进行打包，并将分类器分成.features_和.clf_两部分来进行构建：


![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/辅助分类器.PNG?versionId=CAEQFRiBgIC7w62XyhciIGNkN2E5ZGRiMjlmZTQxYzY5YmY1ODA3YWI0YjYyNDYx)

In [20]:
class AuxClf(nn.Module):
    def __init__(self,in_channels:int,num_classes:int,**kwargs) -> None:
        super().__init__()
        self.feature_ = nn.Sequential(nn.AvgPool2d(kernel_size=5,stride=3),
                                    # nn.Conv2d(in_channels,128,1),
                                    # nn.ReLU(inplace=True) nn.Sequential中须写明激活函数
                                    # 为什么此处不写？ ---> 因此这里是BasicConv2d() 已经包含了激活函数
                                    BasicConv2d(in_channels,128,kernel_size=1))
        self.clf_ = nn.Sequential(nn.Linear(4*4*128,1024),
                                  nn.ReLU(inplace=True),
                                  nn.Dropout(p=0.7),
                                  nn.Linear(1024,num_classes),  # 不一定要输出1000个分类
                                  # nn.Softmax(dim=1)   假设使用的损失函数为交叉熵函数
                                    )

    def forward(self,x):
        x = self.feature_(x)
        x = x.view(-1,128*4*4)
        output = self.clf_(x)
        return output

In [21]:
#————————————————————————————测试辅助分类器————————————————————————————
AuxClf(512,1000)


AuxClf(
  (feature_): Sequential(
    (0): AvgPool2d(kernel_size=5, stride=3, padding=0)
    (1): BasicConv2d(
      (conv): Sequential(
        (0): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
  )
  (clf_): Sequential(
    (0): Linear(in_features=2048, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.7, inplace=False)
    (3): Linear(in_features=1024, out_features=1000, bias=True)
  )
)

- 建立GoogLeNet的架构

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/GoogleNet.PNG?versionId=CAEQFRiBgIDNw62XyhciIDA0ZDViOWM5YjYyNDRlZmJiMmMzNWVjOTZlNjk5YmMw)

In [29]:
# conv1的输入为image类型，通道为3
# 计算conv1的padding (224+2p - 7)/2 +1 = 112.5 --> p = 3
# maxpool1 (112-3)/2+1= 55.5 卷积和池化都是自动向下去成,可以设置ceil_model=True设置为向上取整

In [41]:
#————————————————————————————GoogLeNet的架构———————————————————————————
class GoogLetNet(nn.Module):
    def __init__(self,num_classes:int = 1000,blocks=None) -> None:
        super().__init__()
        
        if blocks is None:
            blocks = [BasicConv2d, Inception, AuxClf]  # 
        conv_block = blocks[0]
        Inception_block = blocks[1]
        AuxClf_block = blocks[2]

        # block1 
        self.conv1 = conv_block(3,64,kernel_size=7,stride=2,padding=3)  # conv1的输入为image类型，通道为3
        # 计算conv1的padding (224+2p - 7)/2 +1 = 112.5 --> p = 3     
        self.maxpool1 = nn.MaxPool2d(3,stride=2,ceil_mode=True)  #  重叠池化
        # maxpool1 (112-3)/2+1= 55.5 卷积和池化都是自动向下去成,可以设置ceil_model=True设置为向上取整

        # block2
        self.conv2 = conv_block(64,64,kernel_size=1)  # 1x1的卷积层并没有改变特征的尺寸，只是进行卷积操作
        self.conv3 = conv_block(64,192,kernel_size=3,padding= 1)
        self.maxpool2 = nn.MaxPool2d(3,stride=2,ceil_mode=True)

        # block3有2个Inception块
        self.in3a = Inception_block(192,64,96,128,16,32,32)
        self.in3b = Inception_block(256,128,128,192,32,96,64)
        self.maxpool3 = nn.MaxPool2d(3,stride=2,ceil_mode=True)

        # block 4 5个Inception块
        self.in4a = Inception_block(480,192,96,208,16,48,64)
        self.in4b = Inception_block(512,160,112,224,24,64,64)
        self.in4c = Inception_block(512,128,128,256,24,64,64)
        self.in4d = Inception_block(512,112,144,288,32,64,64)
        self.in4e = Inception_block(528,256,160,320,32,128,128)
        self.maxpool4 = nn.MaxPool2d(3,stride=2,ceil_mode=True)

         # block5
        self.in5a = Inception_block(832,256,160,320,32,128,128)
        self.in5b = Inception_block(832,384,192,384,48,128,128)

        #clf
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))   # 自适应得池化层:只需填需要的特征图尺寸是多少
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024,num_classes)

        # 辅助分类器
        self.aux1 =AuxClf_block(512,num_classes)  # 4a后面的辅助分类器 4a输出512
        self.aux2 = AuxClf_block(528,num_classes)  # 4d后面的辅助分类器

    
    def forward(self,x):
        # block 1
        x = self.maxpool1(self.conv1(x))

        # block 2 
        x = self.maxpool2(self.conv3(self.conv2(x)))

        # block3
        x = self.in3a(x)
        x = self.maxpool3(self.in3b(x))

        # block 4 
        x = self.in4a(x)
        aux1 = self.aux1(x)

        x = self.in4b(x)
        x = self.in4c(x)
        x = self.in4d(x)
        aux2 = self.aux2(x)

        x = self.in4e(x)
        x = self.maxpool4(x)


        # block 5
        x = self.in5a(x)
        x = self.in5b(x)

        # clf
        x = self.avgpool(x) # 全局平均池化以后特征尺寸为1X1 
        x = torch.flatten(x,1)
        x = self.dropout(x)
        x = self.fc(x)

        return x , aux1, aux2  # 返回主分类器和两个辅助分类器的记过

In [42]:
#————————————————————————————测试GoogLeNet————————————————————————————
data = torch.ones(10,3,224,244)

# 实例化
net = GoogLetNet()

fc2, fc1 ,fc0 = net(data)

for i in [fc2, fc1, fc0]:
    print(i.shape)

# 返回1000个线性层的结果
# 接下来应该放入损失函数

torch.Size([10, 1000])
torch.Size([10, 1000])
torch.Size([10, 1000])


In [ ]:
summary(net,(10,3,224,224),device='cpu')

# 这个层次结构中至少有三层，最内部的是普通卷积层构成的分枝branch，

Layer (type:depth-idx)                        Output Shape              Param #
GoogLeNet                                     [10, 1000]                --
├─BasicConv2d: 1-1                            [10, 64, 112, 112]        --
│    └─Sequential: 2-1                        [10, 64, 112, 112]        --
│    │    └─Conv2d: 3-1                       [10, 64, 112, 112]        9,408
│    │    └─BatchNorm2d: 3-2                  [10, 64, 112, 112]        128
│    │    └─ReLU: 3-3                         [10, 64, 112, 112]        --
├─MaxPool2d: 1-2                              [10, 64, 56, 56]          --
├─BasicConv2d: 1-3                            [10, 64, 56, 56]          --
│    └─Sequential: 2-2                        [10, 64, 56, 56]          --
│    │    └─Conv2d: 3-4                       [10, 64, 56, 56]          4,096
│    │    └─BatchNorm2d: 3-5                  [10, 64, 56, 56]          128
│    │    └─ReLU: 3-6                         [10, 64, 56, 56]          --
├─BasicConv2

In [ ]:
#————————————————————————————使用参数depth————————————————————————————

summary(net,(10,3,224,224),device='cpu',depth=1)

Layer (type:depth-idx)                        Output Shape              Param #
GoogLeNet                                     [10, 1000]                --
├─BasicConv2d: 1-1                            [10, 64, 112, 112]        9,536
├─MaxPool2d: 1-2                              [10, 64, 56, 56]          --
├─BasicConv2d: 1-3                            [10, 64, 56, 56]          4,224
├─BasicConv2d: 1-4                            [10, 192, 56, 56]         110,976
├─MaxPool2d: 1-5                              [10, 192, 28, 28]         --
├─Inception: 1-6                              [10, 256, 28, 28]         164,064
├─Inception: 1-7                              [10, 480, 28, 28]         389,376
├─MaxPool2d: 1-8                              [10, 480, 14, 14]         --
├─Inception: 1-9                              [10, 512, 14, 14]         376,800
├─AuxClf: 1-10                                [10, 1000]                3,188,840
├─Inception: 1-11                             [10, 512, 14, 14

### 2.ResNet

即便算法已经基本达到人类水平，但有两个瓶颈一直没有被突破：  
1. 网络能够达到的最大深度依然很浅，VGG是19层，GoogLeNet也没有超过25层  
2. 深度网络的训练难度太大，虽然强行堆叠卷积层或inception让网络加深非常容易，但加深后的网络往往收敛困难，损失很高，精度很低。


- **训练很深的神经网络比创造很深的神经网络架构难得多**  
  - 网络深度加深，精度却在降低，这种现象在深度网络的训练中被称为“退化”（degradation），退化现象的存在导致深度网络在实际任务上的表现常常会不如浅层网络。
  - 深层网络中的函数关系本质上就比浅层网络中的函数关系更复杂、更难拟合，因此深层网络本质上就比浅层网络更难优化和训练。


>在现有优化算法、优化思路下，在浅层网络后增加恒等函数很可能就是最优的（optimal）加深网络深度的方式。



残差网络ResNet基本思想：假设增加深度用的最优结构就是恒等函数，利用恒等函数的性质，将用于加深网络深度的结构向更容易拟合和训练的方向设计。

具体怎么操作呢？VGG用来加深网络的结构是重复的卷积层，GoogLeNet用来加深网络的结构是Inception块。  
而在残差网络中，这个结构块是“残差块”（Residual unit），也可以译作残差单元。在残差网络中,我们将众多残差单元与普通卷积层串联，以实现“在浅层网络后堆叠某种结构、以增加深度”的目的。


无论残差单元拥有怎样的结构，它一定也存在输入值$x$和必须拟合的关系$H(x)$。假设我们使用$F(x)$来表示输入值$x$与输出的函数关系$H(x)$之间的差异，则有$F(x)=H(x)-x$，此时F(x)就是残差（Residual）。
- 拟合效果较好时，$F(x)$取值为0，拟合0与$F(x)$的关系，比拟合一个未知的函数$F(x)$与$H(x)$的关系要容易

![](https://gitee.com/bravojimoon/note-picture/raw/master/torch/%7DQX%7BX09NSKQ2%5DU%5DOD3_2%5DFV.png)



- 残差网络的优势
  - 残差单元几乎实现了0负担增加深度。首先，跳跃连接不带有任何参数，普通卷积层的结构也不复杂，因此残差块的增加不会给模型带来太多额外的参数负担。同时，由于残差单元比普通网络更容易训练，并且在理论上能够保持网路的精度，因此残差网络的深度可以大幅增加
  - 残差单元还可以大幅加速训练和运算速度
    - 残差单元可以在上层卷积层还未训练的时候就迅速将数据信息传递到下一层中。
    - 因为卷积层接近于恒等函数，在对残差网络进行反向传播时，梯度也可以更快速地通过跳跃链接从后往前传递。



现在我们就来复现一下残差网络，我们将创造一个通用的类，在这个类上输入相关的参数，就可以实现上图中展现的五种残差网络。
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)


和复现GoogLeNet时一样，我们先从简单的、可以打包的元素开始定义。残差网络中的卷积层虽然变化多端，但其实只有两种：3x3卷积层与1x1卷积层，并且我们知道，每个卷积层后面都需要跟上BN层，而BN层上可以完成参数初始化。我们就从这里开始写：



In [ ]:
#  basicconv - conv2d + BN + ReLU (-> conv3x3, conv1x1 )
# Residual Unit, Bottleneck

In [43]:
# 导入需要的库
import torch
import torch.nn as nn
from typing import Type,Union,List,Optional
from torchinfo import summary

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/35.PNG?versionId=CAEQFRiBgID07vWryhciIDRhYTkwODA1MDU0YjRmNWQ5ZTk2NzAxMjdkYzE2MGZm)

In [44]:
#————————————————————————————定义3x3的卷积————————————————————————————
# 大部分的3x3卷积都不改变数据结构
def conv3x3(in_,out_,stride=1,initialzero=False):

    bn = nn.BatchNorm2d(out_)
    #需要进行判断：要对BN进行0初始化吗？
    #最后一层就初始化,不是最后一层就不改变gamma和beta
    
    if initialzero == True:
        nn.init.constant_(bn.weight,0)  # 修改bn层的weight
    return nn.Sequential(nn.Conv2d(in_,out_,kernel_size=3,padding=1,stride=stride,bias=False),
                 bn)

In [45]:
# 测试
conv3x3(2,10)

Sequential(
  (0): Conv2d(2, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [46]:
#————————————————————————————定义1x1的卷积————————————————————————————
# 和3x3类似，只是kernel_size和padding不同

def conv1x1(in_,out_,stride=1,initialzero=False):
    bn = nn.BatchNorm2d(out_)
    #需要进行判断：要对BN进行0初始化吗？
    #最后一层就初始化,不是最后一层就不改变gamma和beta
    if initialzero == True:
        nn.init.constant_(bn.weight,0)  # 修改bn层的weight
    return nn.Sequential(nn.Conv2d(in_,out_,kernel_size=1,padding=0,stride=stride,bias=False),
                 bn)

In [47]:
# 执行0初始化
conv1x1(2,10,1,True)[1].weight


Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

- 残差单元类
  - 两个部分的并联 -> 卷积和跳跃连接
  - 一个残差单元中只包含两个卷积层和一个加和功能。

1. 初始化  
&emsp;&emsp;初始化只会发生在每个残差单元最后一个卷积层的bn层上，因此我们将最后一个层的参数initialzero设置为
True，其他地方不做修改，因此其他卷积层中的0初始化功能并未开启。
2. 步长  
&emsp;&emsp;残差网络使用stride=2的卷积层来给特征图降维，并且每次降维都发生在layers与layers之间。
3. 特征图数量的变化  
&emsp;&emsp;在使用残差单元的浅层残差网络里，每当我们利用步长=2来缩小特征图尺寸，特征图的数量也会翻倍。

In [74]:
#————————————————————————————定义：残差单元类————————————————————————————
# 
class ResidualUnit(nn.Module):
    def __init__(self,out_:int,stride1 :int =1,
                in_: Optional[int] = None   # 添加参数in_ 并不影响后面代码
                ) -> None:
        super().__init__()
        
        self.stride1 = stride1
        # 当特征图尺寸需要缩小时，卷积层的输出特征图数量out等于输入特征图in的2倍
        # 不需要缩小时, out = in
        if stride1 != 1:
            in_ = int(out_ / 2)
        else:
            in_ = out_


        # stride1是否等于2呢？如果等于 2-特征图尺寸会发生变化
        # 需要在跳跃连接上增加1x1卷积层来调整特征图尺寸
        # 如果stride1等于1,则说明都不需要做

        # 拟合部分，输入F(x)
        self.fit_ = nn.Sequential(conv3x3(in_,out_,stride=stride1),
                                  nn.ReLU(inplace=True),
                                  conv3x3(out_,out_,initialzero=True)  # 最后一个卷积层一定使用bn层初始化
                                  )

        # 跳跃连接，输出1x1卷积核之后的x
        self.skipconv = conv1x1(in_,out_,stride=stride1)

        # 单独定义放在H(x)之后来使用的激活函数ReLU
        self.relu = nn.ReLU(inplace=True)

    def forward(self,x):
        fx = self.fit_(x)   # 拟合结果
        if self.stride1 != 1:    # 跳跃连接
            x = self.skipconv(x) 

        hx = self.relu(fx+x)
        return hx

In [49]:
#————————————————————————————测试：残差单元————————————————————————————

data = torch.ones(10,64,56,56) #特征图尺寸64x64，输入特征图数量64
conv3_x_18_0 = ResidualUnit(128,stride1=2) #特征图尺寸折半，特征图数量加倍
conv3_x_18_0(data).shape

conv2_x_18_0 = ResidualUnit(64) #特征图尺寸不变，特征图数量也不变
conv2_x_18_0(data).shape

#你是否注意到in_的存在？
ru = ResidualUnit(64,in_ = 64) #输入in_不影响ResidualUnit的任何行为
ru(data).shape

torch.Size([10, 128, 28, 28])

torch.Size([10, 64, 56, 56])

TypeError: ResidualUnit.__init__() got an unexpected keyword argument 'in_'

- 瓶颈结构


![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)

- in_:Optional[int] 选填参数

In [53]:
class Bottleneck(nn.Module):
    def __init__(self,middle_out,
                stride1 = 1,
                in_:Optional[int]=None) -> None:
        """
        in_：输入瓶颈结构的特征图数量，仅在conv1之后紧跟的瓶颈结构才进行填写。其他时候不填写。
       stride1：第一个卷积层/跳跃连接中1x1卷积层的步长
        """
        super().__init__()
        self.stride1 = stride1

        # 利用一个参数middle_out 表达了2个参数 in和out
        # 最终的数据量 = 中间输出量4倍
        out_ = 4 *middle_out

        # 我希望使用选填参数in_来帮助我们区别这个架构是不是在conv1之后
        # 如果不是紧跟在conv1后，就不填写in_ ; 如果是 就填写in_ = 64 
        if in_ == None:
        # 是否需要将特征图的尺寸缩小的场合吗？
        # conv2_x - conv_3x - conv_4x - conv_5x 相互链接的时候
        # 每次都需要将特征图的尺寸折半，同时卷积层上的middle_out = 1/2 in_
        #需要缩小特征图，则输入量 = 中间输出量 * 2  
            #不需要缩小特征图，则输入量 = 中间输出量 * 4
            if stride1 != 1:  # 缩小特征图的场合，这个瓶颈结构是每个layers的第一个瓶颈结构
                in_ = middle_out *2
            else:             # 不缩小特征图的场合，是在第一个瓶颈结构之后的重复结构
                in_ = middle_out * 4


        
        self.fit_ = nn.Sequential(conv1x1(in_,middle_out,stride=stride1),
                                  nn.ReLU(inplace=True),
                                  conv3x3(middle_out,middle_out),
                                  nn.ReLU(inplace=True),
                                  conv1x1(middle_out,out_,initialzero=True))

        self.skipconv = conv1x1(in_,out_,stride=stride1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self,x):
        fx = self.fit_(x)
        # 跳跃连接
        x = self.skipconv(x)
        hx = self.relu(x+fx)
        return hx

In [54]:
#————————————————————————————测试：瓶颈结构 - conv1后紧跟的第一个瓶颈结构————————————————————————————

data1 = torch.ones(10,64,56,56) #特征图尺寸56x56，特征图数量64
#是conv1后紧跟的第一个瓶颈结构
conv2_x_101_0 = Bottleneck(in_ = 64, middle_out = 64) 
conv2_x_101_0(data1).shape

torch.Size([10, 256, 56, 56])

In [55]:
#—————————测试：瓶颈结构 -不是conv1后紧跟的第一个瓶颈结构，但是需要缩小特征图尺寸——————————————

data2 = torch.ones(10,256,56,56)
#不是conv1后紧跟的第一个瓶颈结构，但是需要缩小特征图尺寸
conv3_x_101_0 = Bottleneck(middle_out = 128, stride1=2)
conv3_x_101_0(data2).shape #输出翻2倍并缩小特征图尺寸至一半

torch.Size([10, 512, 28, 28])

In [56]:
#—————————测试：瓶颈结构 -不是conv1后紧跟的第一个瓶颈结构，也不需要缩小特征图尺寸——————————————


data3 = torch.ones(10,512,28,28)
#不是conv1后紧跟的第一个瓶颈结构，也不需要缩小特征图尺寸
conv3_x_101_1 = Bottleneck(128)
conv3_x_101_1(data3).shape #输出不变，特征图尺寸也不变


torch.Size([10, 512, 28, 28])

不难发现，虽然每个类内部的逻辑需要进行一些梳理，但我们完成的类只有3个参数：这个块中的输出的特征图数量/中间输出量，这个块中第一个卷积层的步长，以及选填的输入特征图数量。

对于同一个layer，残差块中的输出特征图数目 = 瓶颈架构中的中间输出量，而步长其实隐性地决定了这个block在架构中是否位于需要降低特征图尺寸的位置。现在，我们需要将这两个类打包到一个更高级的类中，用来生成每个layers中所有的blocks。

*class Bottleneck(middle_out, stride1, in_(optional))*  
*class ResidualUnit(out_, stride1, in_(optional))*

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)


观察架构图。残差网络的每个layers中都存在大量重复的元素，在深层残差网络中，conv4_x中甚至将瓶颈架构重复了30次以上，如果能够使用代表数量的参数和for循环来对重复的部分进行控制，我们就可以大幅提高生成网络的效率。

In [59]:
# 34 layer - conv3_x
ru0 = ResidualUnit(out_=1215,stride1=2)  
ru1 = ResidualUnit(out_=128)
ru2 = ResidualUnit(out_=128)
ru4 = ResidualUnit(out_=128)

#————————————————————————————上述代码可以用循环实现————————————————————————————
ru0_ = ResidualUnit(out_=1215,stride1=2)  

num_blocks_conv3x = 4

for i in range(num_blocks_conv3x - 1):
    print(ResidualUnit(out_=128))

ResidualUnit(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (skipconv): Sequential(
    (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace=True)
)
ResidualUnit(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Seque

In [67]:
# 50 layer - conv4_x
bt0 = Bottleneck(middle_out=256,stride1=2)
bt1 = Bottleneck(middle_out=256)
bt2 = Bottleneck(middle_out=256)
# ...
bt5 = num_blocks_conv3x - 1

#————————————————————————————上述代码也可以用循环实现————————————————————————————

num_blocks_conv4x = 6
bt0 = Bottleneck(middle_out=256,stride1=2)
# 在列表中添加第个瓶颈架构快
conv4_x50 = []
conv4_x50.append(Bottleneck(middle_out=256,stride1=1))

for i in range(num_blocks_conv4x - 1):
    conv4_x50.append(Bottleneck(middle_out=256))

print(conv4_x50)



# 考虑打包为一个函数

[Bottleneck(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): ReLU(inplace=True)
    (4): Sequential(
      (0): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (skipconv): Sequential(
    (0): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace=True)
), Bottleneck(
  (fit_): Sequential(
    (0): Sequential(
      (0): Conv2d(102

- 第一个块
  - if 这个块是conv1之后的参数:
    - 需要一个特殊的参数 in_ = 64
  - else: 
    - 第一层需要的独特的参数 stride1 = 2 
- 剩下的块就使用for 循环

In [72]:
#————————————————————————————测试：用于瓶颈结构————————————————————————————
from numpy import append


layers = []
num_blocks = 6
afterconv1 = True  # 是否为conv1后的第一个快

if afterconv1 == True:
    layers.append(Bottleneck(middle_out=64,in_=64))
else:
    layers.append(Bottleneck(middle_out=128,stride1=2))

for i in range(num_blocks - 1):
    layers.append(Bottleneck(middle_out=128))

len(layers)
# layers 包含了6个类

# 不能用于残差单元...因为残差单元中没有参数in_


6

In [77]:
#————————————————————————————用于残差单元————————————————————————————
# 在残差单元定义类时，添加参数in_,并且也将其设置为optional
layers = []
num_blocks = 6
afterconv1 = True  # 是否为conv1后的第一个块


if afterconv1 == True:
    layers.append(ResidualUnit(out_=64,in_=64))
else:
    layers.append(ResidualUnit(out_=64,stride1=2))

for i in range(num_blocks - 1):
    layers.append(ResidualUnit(out_=128))

len(layers)
# layers 包含了6个类

6

- Type[Union[ResidualUnit,Bottleneck]
  - Type[ ]  表示只能填写的就是类
  - Union[ ] 输入多个类，需要用Union连起来

- SyntaxError: non-default argument follows default argument
  - 非默认值的参数跟在了默认值参数的后面 错误
  - 先写没有默认值的参数

In [91]:
# 将一个layers内的全部块打包到一个列表中
# 专门用于生成ResNet中的每一个layers中的函数

def make_layers(block: Type[Union[ResidualUnit,Bottleneck]], # 只能填入类，且只能填入RU BN之一
                middle_out:int,
                num_blocks :int,
                afterconv1: bool=False):

    '''
   构建残差网络中layers的类
    
   block: 架构块的类型，可选ResidualUnit或Bottleneck。依据选择的架构块类型，可判断该残差网络的深浅
   middle_out: ResidualUnit中输出的特征图数目/Bottleneck中的中间输出量，对两个block可混用
   blocks：这个layer中的block数量
   afterconv1：这个layer是否紧接在conv1之后？
   '''
    layers = []
    
    if afterconv1 == True:    # 是否为conv1之后的第一个block
        layers.append(block(middle_out,in_=64))
    else:
        layers.append(block(middle_out,stride1=2))

    for i in range(num_blocks - 1):   # 重复残差单元 / 瓶颈架构
        layers.append(block(middle_out))
    
    return nn.Sequential(* layers)

In [89]:
#————————————————————————————测试函数：make_layers————————————————————————————
layer_34_conv4_x = make_layers(ResidualUnit,256,6,False)
len(layer_34_conv4_x)

6

- 星号解析列表/储存器 `nn.Sequential(* layer_34_conv4_x) `
  - layer_34_conv4_x 是一个列表不能放到 nn.Sequential中

In [90]:
# nn.Sequential(layer_34_conv4_x)  TypeError: list is not a Module subclass

nn.Sequential(* layer_34_conv4_x) 

Sequential(
  (0): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ReLU(inplace=True)
      (2): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (skipconv): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(inplace=True)
  )
  (1): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_

In [94]:
#————————————————————————————4个测试————————————————————————————
# 测试 - 需要分别对残差块 和 瓶颈架构进行测试，并且需要对conv1后的首个架构，以及中间的架构进行测试
#注意检查：输入的数据结构是否正确，网络能否允许正确的数据结构输入，输入后产出的结构是否正确，
# 包括特征图尺寸是否变化、特征图数量是否变化，以及一个layers中所包含的blocks数量是否正确


Sequential(
  (0): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ReLU(inplace=True)
      (2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (skipconv): Sequential(
      (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(inplace=True)
  )
  (1): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 64, 56, 56]          --
├─ResidualUnit: 1-1                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-2                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-3                      [10, 64, 56, 56]          78,208
Total params: 234,624
Trainable params: 234,624
Non-trainable params: 0
Total mult-adds (G): 6.94
Input size (MB): 8.03
Forward/backward pass size (MB): 192.68
Params size (MB): 0.89
Estimated Total Size (MB): 201.59

In [96]:
#————————————————————————————测试1:残差块后的首个架构————————————————————————————

# 34层网络，conv2_x,紧跟在conv1后的首页个架构
# 不改变特征图尺寸，且每层的输出都是64
make_layers(ResidualUnit,128,3,afterconv1=True)

conv2_x_34 = make_layers(ResidualUnit,64,3,afterconv1=True)

datashape = (10,64,56,56)

summary(conv2_x_34,datashape,depth=1,device='cpu')


Sequential(
  (0): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ReLU(inplace=True)
      (2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (skipconv): Sequential(
      (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (relu): ReLU(inplace=True)
  )
  (1): ResidualUnit(
    (fit_): Sequential(
      (0): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 64, 56, 56]          --
├─ResidualUnit: 1-1                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-2                      [10, 64, 56, 56]          78,208
├─ResidualUnit: 1-3                      [10, 64, 56, 56]          78,208
Total params: 234,624
Trainable params: 234,624
Non-trainable params: 0
Total mult-adds (G): 6.94
Input size (MB): 8.03
Forward/backward pass size (MB): 192.68
Params size (MB): 0.89
Estimated Total Size (MB): 201.59

In [105]:
#————————————————————————————测试2————————————————————————————
#34层网络，conv4_x，缩小特征图尺寸，且每层的输出翻倍
datashape2 = (10,128,14,14)
conv2_x_34 = make_layers(ResidualUnit,256, 6, afterconv1=False)
summary(conv2_x_34,datashape2,depth=1,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 256, 7, 7]           --
├─ResidualUnit: 1-1                      [10, 256, 7, 7]           919,040
├─ResidualUnit: 1-2                      [10, 256, 7, 7]           1,246,720
├─ResidualUnit: 1-3                      [10, 256, 7, 7]           1,246,720
├─ResidualUnit: 1-4                      [10, 256, 7, 7]           1,246,720
├─ResidualUnit: 1-5                      [10, 256, 7, 7]           1,246,720
├─ResidualUnit: 1-6                      [10, 256, 7, 7]           1,246,720
Total params: 7,152,640
Trainable params: 7,152,640
Non-trainable params: 0
Total mult-adds (G): 3.34
Input size (MB): 1.00
Forward/backward pass size (MB): 26.09
Params size (MB): 27.29
Estimated Total Size (MB): 54.38

In [97]:
#————————————————————————————测试3:瓶颈架构————————————————————————————
# 101层网络，conv2_x,紧跟在conv1后的首页个架构
# 不改变特征图尺寸，且每层的输出都是64


conv2_x_101 = make_layers(Bottleneck,64,3,afterconv1=True)

datashape = (10,64,56,56)

summary(conv2_x_101,datashape,depth=3,device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 256, 56, 56]         --
├─Bottleneck: 1-1                        [10, 256, 56, 56]         --
│    └─Sequential: 2-1                   [10, 256, 56, 56]         --
│    │    └─Sequential: 3-1              [10, 64, 56, 56]          4,224
│    │    └─ReLU: 3-2                    [10, 64, 56, 56]          --
│    │    └─Sequential: 3-3              [10, 64, 56, 56]          36,992
│    │    └─ReLU: 3-4                    [10, 64, 56, 56]          --
│    │    └─Sequential: 3-5              [10, 256, 56, 56]         16,896
│    └─Sequential: 2-2                   [10, 256, 56, 56]         --
│    │    └─Conv2d: 3-6                  [10, 256, 56, 56]         16,384
│    │    └─BatchNorm2d: 3-7             [10, 256, 56, 56]         512
│    └─ReLU: 2-3                         [10, 256, 56, 56]         --
├─Bottleneck: 1-2                        [10, 256, 56, 56]         --

In [102]:
#————————————————————————————测试4：瓶颈架构————————————————————————————
#101层网络，conv4_x，缩小特征图尺寸，且每层的输出翻4倍
datashape3 = (10,512,28,28)
conv4_x_101 = make_layers(Bottleneck,256,23,afterconv1=False)
summary(conv4_x_101,datashape3,depth=1,device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [10, 1024, 14, 14]        --
├─Bottleneck: 1-1                        [10, 1024, 14, 14]        1,512,448
├─Bottleneck: 1-2                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-3                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-4                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-5                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-6                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-7                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-8                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-9                        [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-10                       [10, 1024, 14, 14]        2,167,808
├─Bottleneck: 1-11                       [10, 1024, 14, 14]        2,167,808
├─Bottle

现在我们已经具备了构建layers的能力，可以开始构建自己的残差网络了。定义残差网络的类ResNet可能是所有复现步骤中最简单的一个，它与我们之前熟悉的其他网络的定义方式非常类似。参照架构图，定义残差网络的代码如下：

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/ResNet架构.PNG?versionId=CAEQFRiBgIDQw62XyhciIDUzZmRhZGJhNThjNTQ3NzhhZjAxOGFjYWM1MjYxZmMw)

In [ ]:
# 建立18层网络 layers = [2,2,2,2]
# 建立32层网络： layes = [3,4,6,3]

In [108]:
class ResNet(nn.Module):
    def __init__(self,block:Type[Union[ResidualUnit, Bottleneck]],
                layers:List[int],
                num_classes:int) -> None:

        '''
       block：要使用的用来加深深度的基本架构是？可以选择残差单元或瓶颈结构，两种都带有skip connection
       layers：列表，每个层里具体有多少个块呢？可参考网络架构图。例如，34层的残差网络的layers = [3,4,6,3]
       num_classes：真实标签含有多少个类别？
       '''
    
        super().__init__()

        # layer 1:卷积+池化
        self.layer1 = nn.Sequential(nn.Conv2d(3,64,7,stride=2,padding=3,bias=False),
                                    nn.BatchNorm2d(64),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(3,stride=2,ceil_mode=True))
        # layer 2 - 5: 残差块 / 瓶颈架构
        self.layer2_x = make_layers(block,64,layers[0],afterconv1=True) 
        self.layer3_x = make_layers(block,128,layers[1]) 
        self.layer4_x = make_layers(block,256,layers[2]) 
        self.layer5_x = make_layers(block,512,layers[3]) 

        # 全局平均池化
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        # 分类 - 使用瓶颈架构的深度残差网络最终的输出值包含2048个像素,而浅层残差网络最终的输出值包含512个像素
        if block == ResidualUnit:
            self.fc = nn.Linear(512,num_classes)
        else:
            self.fc = nn.Linear(2048,num_classes)

    def forward(self,x):
        x = self.layer1(x)   # layer1,普通卷积+池化输出
        x = self.layer5_x(self.layer4_x(self.layer3_x(self.layer2_x(x))))
        x = self.avgpool(x)  # 特征图的尺寸为1*1*sample
        x = torch.flatten(x,1)
        x = self.fc(x)

In [109]:
#————————————————————————————测试:建立34层的残差网络————————————————————————————
datashape = (10,3,224,224)


res34 = ResNet(ResidualUnit,[3,4,6,3],1000)
summary(res34,datashape,depth=2,device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        --                        --
├─Sequential: 1-1                             [10, 64, 56, 56]          --
│    └─Conv2d: 2-1                            [10, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [10, 64, 112, 112]        128
│    └─ReLU: 2-3                              [10, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [10, 64, 56, 56]          --
├─Sequential: 1-2                             [10, 64, 56, 56]          --
│    └─ResidualUnit: 2-5                      [10, 64, 56, 56]          78,208
│    └─ResidualUnit: 2-6                      [10, 64, 56, 56]          78,208
│    └─ResidualUnit: 2-7                      [10, 64, 56, 56]          78,208
├─Sequential: 1-3                             [10, 128, 28, 28]         --
│    └─ResidualUnit: 2-8                      [10, 128, 28, 28]         230,144

In [113]:
#————————————————————————————测试：建立101层的残差网络————————————————————————————
res101 = ResNet(Bottleneck, layers=[3,4,23,3], num_classes=1000)

summary(res101,datashape,depth=2,device="cpu")


# 101层只有7kw+个参数


Layer (type:depth-idx)                        Output Shape              Param #
ResNet                                        --                        --
├─Sequential: 1-1                             [10, 64, 56, 56]          --
│    └─Conv2d: 2-1                            [10, 64, 112, 112]        9,408
│    └─BatchNorm2d: 2-2                       [10, 64, 112, 112]        128
│    └─ReLU: 2-3                              [10, 64, 112, 112]        --
│    └─MaxPool2d: 2-4                         [10, 64, 56, 56]          --
├─Sequential: 1-2                             [10, 256, 56, 56]         --
│    └─Bottleneck: 2-5                        [10, 256, 56, 56]         75,008
│    └─Bottleneck: 2-6                        [10, 256, 56, 56]         136,448
│    └─Bottleneck: 2-7                        [10, 256, 56, 56]         136,448
├─Sequential: 1-3                             [10, 512, 28, 28]         --
│    └─Bottleneck: 2-8                        [10, 512, 28, 28]         379,3

到这里，我们就已经复现了整个残差网络。即便ResNet现在可以达到的深度非常深，但从参数和计算量的角度来看，它并不算是“巨型”的模型。

从模型效果来看，残差网络毫无疑问是现有的最顶尖的模型之一，几乎所有大型数据集的跑分榜单前几名都是残差网络占据。除了我们已经学习的基本网络，残差网络还有许多有效、强大的变体，如ResNeXt（在残差网络上加入了并联结构），WideResNet（目前为止最强大的模型）等。